In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
import numpy as np

import warnings
warnings.filterwarnings("ignore")


In [14]:

# Load the dataset
data = pd.read_csv('train.csv') 
data['Order Date'] = pd.to_datetime(data['Order Date'], format='%d/%m/%Y')

# filter for past year
latest_date = data['Order Date'].max()
one_year_ago = latest_date - pd.DateOffset(years=1)
filtered_data = data[data['Order Date'] >= one_year_ago]

# process sales, cumulative sales sorted by sales, and corresponding cumulative contribution
sales_by_product = filtered_data.groupby(['Sub-Category', 'Category'])['Sales'].sum().reset_index()
total_sales = sales_by_product['Sales'].sum()
sales_by_product = sales_by_product.sort_values(by='Sales', ascending=False)
sales_by_product['Cumulative Sales'] = sales_by_product['Sales'].cumsum()
sales_by_product['Cumulative Contribution'] = (sales_by_product['Cumulative Sales'] / total_sales) * 100

#classification of categories A, B, C
def classify_product(contribution):
    if contribution <= 80:
        return 'A'
    elif contribution <= 95:
        return 'B'
    else:
        return 'C'

sales_by_product['Classification'] = sales_by_product['Cumulative Contribution'].apply(classify_product)

#categorized sub-categories
sales_by_product

,Sub-Category,Category,Sales,Cumulative Sales,Cumulative Contribution,Classification
13,Phones,Technology,104988.0340,104988.0340,14.509977,A
5,Chairs,Furniture,93362.0590,198350.0930,27.413174,A
3,Binders,Office Supplies,71932.7070,270282.8000,37.354706,A
14,Storage,Office Supplies,68710.5340,338993.3340,46.850914,A
6,Copiers,Technology,62899.3880,401892.7220,55.543988,A
16,Tables,Furniture,60893.5425,462786.2645,63.959841,A
0,Accessories,Technology,58240.9700,521027.2345,72.009093,A
11,Machines,Technology,43544.6750,564571.9095,78.027228,A
1,Appliances,Office Supplies,40212.7220,604784.6315,83.584868,B
4,Bookcases,Furniture,30024.2797,634808.9112,87.734404,B
